In [1]:
import os
import openai
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import AzureSearch
from dotenv import load_dotenv
load_dotenv('.env')


True

In [2]:
embeddings = OpenAIEmbeddings()

# Connect to Azure Cognitive Search
acs = AzureSearch(azure_search_endpoint=os.getenv('SEARCH_SERVICE_NAME'),
                 azure_search_key=os.getenv('SEARCH_API_KEY'),
                 index_name=os.getenv('SEARCH_INDEX_NAME'),
                 embedding_function=embeddings.embed_query)

In [3]:
from langchain.document_loaders import CSVLoader

loader = CSVLoader("wine-ratings.csv")
documents = loader.load()

In [4]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [5]:
acs.add_documents(documents=docs)

['ZWQxZGI0MWUtMmYwZi00MjI2LTgzMTctMTk1NTNkYTM4Y2Vi',
 'ZTkwYWE0YzAtY2MxYi00NGRmLWJiZmUtZDVkYjRiMjhiZWQw',
 'MGQ1YzRmZTgtZTBlOC00YTMyLWE2ZmYtNTQzMjhlNzc3NWEy',
 'OTBhYmIzNDItYTQ2Ny00MGY0LWFmMTYtNTUzMzRmYjlmY2Ew',
 'NTA3YWQ0ZjYtOGE1ZC00NjM4LTkzZTMtNjAxMWE1MzUwNTg0',
 'OTZkYWMzNTctNzg1YS00MWY2LWJkOGMtMTI3Y2JmYTBiMzNk',
 'YjZhMmZkOTYtMTI4Zi00MjhiLTg2OWUtNzU0M2FiYTQwNmJl',
 'ZjkzYzFkNDMtODNmNi00MWMxLWI5NjUtYTRjODE0MmZmMmM4',
 'ZmRjNjE3MjAtMDFiZC00Y2I4LTgyMDctNzJkMWNhMjQ3YWYw',
 'YmNhNTMzYWYtOTRkMi00NGM3LWFkNTItNDA2YzkwNzkxYzBh',
 'ODlkM2I5MjctNDg2Ni00NDUzLWIzMDUtZWJiOGM3MDcxOWFk',
 'ODZlY2ZiMzItOTM1Yy00OTc3LWE1NjItNWFlOWYwZmZmY2Y3',
 'NWE0MGVhZWItMTJmMC00ZjA0LTg2YmMtMjcwYjdlYmExODBj',
 'MDUxOGZjMDItMzY1Mi00ZmY5LWE1YjItZjJmYjFlMDAyODlj',
 'NGMyNGFhYjMtZDM4Zi00ZTAzLTk4ZjItMzUxYzkzOGFmODE5',
 'MmYzMDkyNDgtM2IwYS00ZWQyLTgwNDEtODExZDUzODc5ODMx',
 'MTI3MDNjMzktYWM4Ni00NGUyLWIwYWEtYjRlYmFiNjgyMTQy',
 'MWY5ZjFhYmUtZjQwMC00Y2I5LWE1YTAtZWVlZWJmOTk3ZWI2',
 'YjFkYTEzYjktYWQ5Ni00ZjViLWFiNGQtYTk2ODZkYWIx

In [6]:
docs = acs.similarity_search_with_relevance_scores(
    query="What is the best red wine available in the market?",
    k=5,
)
print(docs[0][0].page_content)
print(dir(docs[0][0]))

: 11
name: 14 Hands Cabernet Sauvignon 2015
grape: 
region: Columbia Valley, Washington
variety: Red Wine
rating: 89.0
notes: The 14 Hands Cabernet Sauvignon is a rich, juicy red that features aromas of dark cherry, black currant, coffee and subtle hints of spice. These flavors are complemented by a touch of spicy oak and emphasized by refined tannins.
['Config', '__abstractmethods__', '__annotations__', '__class__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_n

In [10]:
# openai.api_base = os.getenv("OPENAI_API_BASE")  # Your Azure OpenAI resource's endpoint value.
openai.api_key = os.getenv("OPENAI_API_KEY")
# openai.api_type = "azure"
# openai.api_version = "2023-05-15" 
messages=[
    {"role": "system", "content": "Asisstant is a chatbot that helps you find the best wine for your taste."},
    {"role": "user", "content": "What is the best wine in Oregon above 92 points?"},
    {"role": "assistant", "content": docs[0][0].page_content}
]

response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
)
from pprint import pprint
pprint(response)
print(response['choices'][0]['message']['content'])

<OpenAIObject chat.completion id=chatcmpl-9vFTUDC8ynFHC8CgLeFiigXrC6dRO at 0x124957290> JSON: {
  "id": "chatcmpl-9vFTUDC8ynFHC8CgLeFiigXrC6dRO",
  "object": "chat.completion",
  "created": 1723431208,
  "model": "gpt-3.5-turbo-0125",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "I recommend trying the 14 Hands Cabernet Sauvignon 2015 from Columbia Valley, Washington. It has a rating of 89 points and offers rich flavors of dark cherry, black currant, coffee, and spice, with refined tannins.",
        "refusal": null
      },
      "logprobs": null,
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 142,
    "completion_tokens": 52,
    "total_tokens": 194
  },
  "system_fingerprint": null
}
I recommend trying the 14 Hands Cabernet Sauvignon 2015 from Columbia Valley, Washington. It has a rating of 89 points and offers rich flavors of dark cherry, black currant, coffee, and spice, with refined tannins.
